In [1]:
!pip install vllm
from vllm import LLM, SamplingParams

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 M

INFO 06-01 08:40:26 [__init__.py:243] Automatically detected platform cuda.


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
from datasets import load_dataset
import random
import numpy as np
import csv
from huggingface_hub import login
login('enter your token')

In [3]:
def main():
    ############## Set Up ##############
    torch.manual_seed(0)
    random.seed(0)

    max_new_tokens = 256    # Number of new tokens to generate
    device_count = torch.cuda.device_count()
    tensor_parallel_size = device_count if device_count > 1 else 1

    model_name = "meta-llama/Llama-3.2-3B-Instruct"
    llm = LLM(
        model=model_name,
        dtype="float16",
        tensor_parallel_size=tensor_parallel_size,
        max_model_len=4096,                 # <-- added to avoid KV cache overflow
        gpu_memory_utilization=0.95,
        max_num_seqs=2*4,
        max_num_batched_tokens=8192*4,# <-- added to better use GPU memory

    )
    #####################################

    warmup_prompt = "Explain what AI is."
    sampling_params = SamplingParams(
        max_tokens=max_new_tokens,
        temperature=0.0  # deterministic for consistent speed
    )

    # === Warm-up phase ===
    print("Starting warm-up...")
    for i in tqdm(range(5), desc="Warm Up..."):
        outputs = llm.generate([warmup_prompt], sampling_params)

    # === Test phase ===
    prompt = "How to learn a new language?"
    tputs = []
    time_record = []
    for _ in tqdm(range(10), desc="Test Inference"):
        torch.cuda.synchronize()
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()

        outputs = llm.generate([prompt], sampling_params)

        end_event.record()
        torch.cuda.synchronize()
        elapsed_ms = start_event.elapsed_time(end_event)
        elapsed_sec = elapsed_ms / 1000
        tput = max_new_tokens / elapsed_sec
        time_record.append(elapsed_sec)
        tputs.append(tput)

    # Get response (first output)
    response = outputs[0].outputs[0].text
    sorted_tputs = np.sort(tputs)[2:-2]
    org_tput = np.mean(sorted_tputs)

    print(f'Prompt: {prompt}\nResponse: {response}\n')
    print(f'Time Record: {time_record}')
    print(f'Throughput Record: {tputs} toks/s\n')
    print(f'Throughput: {org_tput} toks/s')
    import csv
    # Save results to CSV
    rounded_tput = round(org_tput, 1)
    with open("result.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Id", "value"])
        writer.writerow([1, rounded_tput])  # Only throughput here

if __name__ == '__main__':
    main()

INFO 06-01 07:47:30 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-01 07:47:30 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-01 07:47:30 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

WARNING 06-01 07:47:32 [config.py:3135] Casting torch.bfloat16 to torch.float16.
INFO 06-01 07:48:02 [config.py:793] This model supports multiple tasks: {'classify', 'embed', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 06-01 07:48:02 [arg_utils.py:1583] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 06-01 07:48:02 [config.py:2155] max_num_batched_tokens (65536) exceeds max_num_seqs* max_model_len (32768). This may lead to unexpected behavior.
INFO 06-01 07:48:02 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.0.1) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disabl

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

...

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: How to learn a new language?
Response:  Here are some effective ways to learn a new language:
1.  Immersion: Surround yourself with the language you want to learn. Listen to music, watch TV shows and movies, read books and newspapers, and speak with native speakers.
2.  Language learning apps: There are many language learning apps available that can help you learn a new language, such as Duolingo, Babbel, and Rosetta Stone.
3.  Language exchange programs: Find a language exchange partner, either in person or online, to practice speaking and listening with a native speaker.
4.  Language courses: Enroll in a language course at a local college or language school, or take online courses.
5.  Flashcards: Create flashcards to help you memorize new vocabulary and grammar rules.
6.  Practice speaking and listening: Practice speaking and listening to improve your pronunciation and comprehension.
7.  Focus on grammar: Learn the grammar rules of the language you want to learn, including v